In [1]:
import pandas as pd
import re
from functools import reduce

In [2]:
cities = ['Atlanta', 'Boston', 'Chicago', 'Detroit', 'Los Angeles', 'Miami', 'New York', 'Seattle']

In [3]:
city = pd.read_csv('city_attributes_.csv', index_col=0)
print(city.columns)
city.columns = ['city', 'country', 'lat', 'lon']
city

Index(['City', 'Country', 'Latitude', 'Longitude'], dtype='object')


,city,country,lat,lon
0,Seattle,United States,47.606209,-122.332069
1,Los Angeles,United States,34.052231,-118.243683
2,Chicago,United States,41.850029,-87.650047
3,Atlanta,United States,33.749001,-84.387978
4,Detroit,United States,42.331429,-83.045753
5,Miami,United States,25.774269,-80.193657
6,New York,United States,40.714272,-74.005966
7,Boston,United States,42.358429,-71.059769


In [4]:
def format_str(x):
    new_str = ''
    for i, c in enumerate(x):
        if c == '"' and (x[i-1] in '{,' or x[i+1] == ':'):
            continue
        new_str += c
    return new_str

In [5]:
format_str(city.to_json(orient='records'))

'[{city:"Seattle",country:"United States",lat:47.606209,lon:-122.332069},{city:"Los Angeles",country:"United States",lat:34.052231,lon:-118.243683},{city:"Chicago",country:"United States",lat:41.850029,lon:-87.650047},{city:"Atlanta",country:"United States",lat:33.749001,lon:-84.387978},{city:"Detroit",country:"United States",lat:42.331429,lon:-83.045753},{city:"Miami",country:"United States",lat:25.774269,lon:-80.193657},{city:"New York",country:"United States",lat:40.714272,lon:-74.005966},{city:"Boston",country:"United States",lat:42.358429,lon:-71.059769}]'

In [6]:
temperature = pd.read_csv('temperature_.csv', index_col=0)
temperature = pd.melt(temperature, id_vars=['date'], value_vars=cities, var_name='city', value_name='temperature')
temperature['temperature'] = temperature['temperature'] - 273.15

In [7]:
humidity = pd.read_csv('humidity_.csv', index_col=0)
humidity = pd.melt(humidity, id_vars=['date'], value_vars=cities, var_name='city', value_name='humidity')

In [8]:
pressure = pd.read_csv('pressure_.csv', index_col=0)
pressure = pd.melt(pressure, id_vars=['date'], value_vars=cities, var_name='city', value_name='pressure')

In [9]:
wind_direction = pd.read_csv('wind_direction_.csv', index_col=0)
wind_direction = pd.melt(wind_direction, id_vars=['date'], value_vars=cities, var_name='city', value_name='wind_direction')

In [10]:
wind_speed = pd.read_csv('wind_speed_.csv', index_col=0)
wind_speed = pd.melt(wind_speed, id_vars=['date'], value_vars=cities, var_name='city', value_name='wind_speed')

In [11]:
meteo = [temperature, humidity, pressure, wind_direction, wind_speed]
meteo = reduce(lambda left,right: pd.merge(left,right,on=['date', 'city']), meteo)
# meteo = meteo[meteo['date'] >= '2016-11-30']
meteo['date'] = meteo['date'].apply(lambda x: x[:-3])
meteo = meteo.groupby(['date', 'city'], as_index=False).mean()
meteo.dropna(inplace=True)
for col in meteo.columns[2:]:
    meteo[col] = meteo[col].astype(int)
print(meteo.shape)
meteo.head()

(278, 7)


,date,city,temperature,humidity,pressure,wind_direction,wind_speed
0,2015-01,Atlanta,4,68,1036,225,2
1,2015-01,Boston,-2,99,1030,235,5
2,2015-01,Chicago,-5,97,1035,218,4
3,2015-01,Detroit,-7,82,1034,214,4
4,2015-01,Los Angeles,9,58,1033,81,0


In [12]:
str_ = format_str(meteo.to_json(orient='records'))
str_

'[{date:"2015-01",city:"Atlanta",temperature:4,humidity:68,pressure:1036,wind_direction:225,wind_speed:2},{date:"2015-01",city:"Boston",temperature:-2,humidity:99,pressure:1030,wind_direction:235,wind_speed:5},{date:"2015-01",city:"Chicago",temperature:-5,humidity:97,pressure:1035,wind_direction:218,wind_speed:4},{date:"2015-01",city:"Detroit",temperature:-7,humidity:82,pressure:1034,wind_direction:214,wind_speed:4},{date:"2015-01",city:"Los Angeles",temperature:9,humidity:58,pressure:1033,wind_direction:81,wind_speed:0},{date:"2015-01",city:"Miami",temperature:20,humidity:82,pressure:1033,wind_direction:163,wind_speed:3},{date:"2015-01",city:"New York",temperature:-4,humidity:78,pressure:1018,wind_direction:230,wind_speed:2},{date:"2015-01",city:"Seattle",temperature:5,humidity:97,pressure:1036,wind_direction:129,wind_speed:1},{date:"2015-02",city:"Atlanta",temperature:3,humidity:65,pressure:1034,wind_direction:251,wind_speed:3},{date:"2015-02",city:"Boston",temperature:-5,humidity:99

In [13]:
len(str_)

29194

In [14]:
meteo.to_csv('meteo_data.csv', index=False)